In [1]:
# !pip install boto3
import boto3

In [2]:
# I have preconfigured aksk using AWS CLI
s3 = boto3.resource('s3')

# create a s3 bucket
try:
    s3.create_bucket(Bucket='datacont-chunying', CreateBucketConfiguration = {
        'LocationConstraint': 'us-west-2'}
    ) 
except Exception as e:
    print (e)

In [3]:
bucket = s3.Bucket("datacont-chunying")

# make this bucket publicly readable
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'QGYPMR86E78PF144',
  'HostId': 'Sz37gB9fd/2cUEgclP+rbfja07/NVV7/ynUu102R4qv54kWFDnNUNuk3nQXZ06t/z3yA6sekf98=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Sz37gB9fd/2cUEgclP+rbfja07/NVV7/ynUu102R4qv54kWFDnNUNuk3nQXZ06t/z3yA6sekf98=',
   'x-amz-request-id': 'QGYPMR86E78PF144',
   'date': 'Wed, 20 Oct 2021 18:14:43 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [4]:
# upload data file into the bucket and make them publicly readable

data_folder = 'data/'
bucket_name = 'datacont-chunying'

file_name = 'experiments.csv'
experiments = open(data_folder+file_name, 'rb')
s3.Object(bucket_name, file_name).put(Body = experiments)
s3.Object(bucket_name, file_name).Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'S3D841CK5Y4VGB0N',
  'HostId': 'vbdoMAG7PR/9VLtDfc4A6zfHsz8G4sZPhESJ93M2/wk6xjf3Q/nIm7Q5HvbjMqy3D37a6Nto+hI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vbdoMAG7PR/9VLtDfc4A6zfHsz8G4sZPhESJ93M2/wk6xjf3Q/nIm7Q5HvbjMqy3D37a6Nto+hI=',
   'x-amz-request-id': 'S3D841CK5Y4VGB0N',
   'date': 'Wed, 20 Oct 2021 18:14:48 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [5]:
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')

In [8]:
table_name = 'DataTable-chunying'

# Create the DynamoDB table.
try:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except Exception as e:
    print (e)
    table = dyndb.Table("DataTable")


In [9]:
# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName=table_name)

# Print out some data about the table.
print(table.item_count)

0


In [11]:
import csv

s3_endpoint = "https://"+bucket_name+".s3.us-west-2.amazonaws.com/"

with open(data_folder+file_name, 'r') as csvfile: 
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf, None)  # skip the headers
    for item in csvf:
        print(item)
        file_item = item[4]
        body = open(data_folder+file_item, 'rb') 
        s3.Object(bucket_name, file_item).put(Body=body)
        md = s3.Object(bucket_name, file_item).Acl().put(ACL='public-read')
        
        url = s3_endpoint + file_item
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[4], 'Temp': item[1],
        'Conductivity' : item[2], 'Concentration' : item[3], 'URL':url}
        
        try: 
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [12]:
# query data
response = table.get_item(
    Key={
        'PartitionKey': '2',
        'RowKey': 'exp2.csv'
    }
)

In [13]:
item = response['Item']
print(item)

{'Temp': '-2', 'RowKey': 'exp2.csv', 'Conductivity': '52.1', 'Concentration': '3.4', 'PartitionKey': '2', 'URL': 'https://datacont-chunying.s3.us-west-2.amazonaws.com/exp2.csv'}


In [14]:
response

{'Item': {'Temp': '-2',
  'RowKey': 'exp2.csv',
  'Conductivity': '52.1',
  'Concentration': '3.4',
  'PartitionKey': '2',
  'URL': 'https://datacont-chunying.s3.us-west-2.amazonaws.com/exp2.csv'},
 'ResponseMetadata': {'RequestId': 'VV9EB9HESQGC37D4K8GJ5KJU67VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 20 Oct 2021 19:05:01 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '211',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'VV9EB9HESQGC37D4K8GJ5KJU67VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3106289144'},
  'RetryAttempts': 0}}